In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/공모전/KTH/data
!ls

/content/drive/MyDrive/공모전/KTH/data
1_8호선_위치정보.csv  tmp.csv			     지하철역.csv
1_9호선_위치정보.csv  weight_logistic.csv	     지하철역정보.csv
9호선_위치정보.csv    가중치_물류량1.csv	     지하철정보.json
IC_정보.csv	      가중치_물류량2_clustering.csv  택배도착량.csv
IC정보.csv	      가중치_물류량.csv		     택배발생량.csv
new_가중치1.csv       지하철역2.csv


In [3]:
import pandas as pd
import numpy as np

In [4]:
subway = pd.read_csv('1_9호선_위치정보.csv')
ic = pd.read_csv('IC정보.csv')

In [5]:
subway.head(2)

,호선,역명,위도,경도
0,1호선,서울,37.55315,126.972533
1,1호선,시청,37.56359,126.975407


In [6]:
ic.head(2)

,IC명,위도,경도
0,안성IC,36.994219,127.156069
1,오산IC,37.142691,127.086371


In [7]:
subway.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 305 entries, 0 to 304
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   호선      305 non-null    object 
 1   역명      305 non-null    object 
 2   위도      305 non-null    float64
 3   경도      305 non-null    float64
dtypes: float64(2), object(2)
memory usage: 9.7+ KB


In [8]:
ic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94 entries, 0 to 93
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   IC명     94 non-null     object 
 1   위도      94 non-null     float64
 2   경도      94 non-null     float64
dtypes: float64(2), object(1)
memory usage: 2.3+ KB


In [12]:
res = pd.DataFrame(columns=['호선', '역명', 'IC명', '거리'])
res

,호선,역명,IC명,거리


In [36]:
x = []
x.extend(['hi', 'bye'])
print(x + ['hello', 1])
print(x)

['hi', 'bye', 'hello', 1]
['hi', 'bye']


In [39]:
!pip install haversine

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [40]:
from haversine import haversine

In [49]:
def cal_distance(df1: pd.DataFrame, df2: pd.DataFrame) -> pd.DataFrame:
  # res = pd.DataFrame(columns=['호선', '역명', 'IC명', '거리'])
  # for row in df1:
  res = []
  for _, row in df1.iterrows():
    new_row = []
    new_row.extend(row[:2])
    for _, row2 in df2.iterrows():
      distance = haversine(row[2:4], row2[1:3], unit='km')
      res.append(new_row + [row2[0], distance])
  return pd.DataFrame(res, columns=['호선', '역명', 'IC명', '거리(km)'])

In [50]:
distance_ic_to_subway = cal_distance(subway, ic)
distance_ic_to_subway

,호선,역명,IC명,거리(km)
0,1호선,서울,안성IC,64.237055
1,1호선,서울,오산IC,46.737168
2,1호선,서울,기흥IC,38.461069
3,1호선,서울,수원신갈IC,33.996703
4,1호선,서울,대왕판교IC,19.140312
...,...,...,...,...
28665,9호선,봉은사,가좌IC,34.105811
28666,9호선,봉은사,서인천IC,33.432129
28667,9호선,봉은사,부평IC,29.766695
28668,9호선,봉은사,부천IC,24.898072


In [51]:
sorted_distance =\
   distance_ic_to_subway.sort_values('거리(km)').reset_index(drop=True)
sorted_distance

,호선,역명,IC명,거리(km)
0,8호선,장지,송파IC,0.431406
1,8호선,복정,송파IC,0.461364
2,8호선,복정,송파IC,0.466299
3,8호선,장지,송파IC,0.568593
4,8호선,모란,성남IC,0.812952
...,...,...,...,...
28665,5호선,김포공항,음성IC,96.502796
28666,9호선,김포공항,음성IC,96.536370
28667,5호선,방화,음성IC,96.887856
28668,5호선,개화산,음성IC,96.891909


In [55]:
sorted_distance.to_csv('tmp_distance.csv', encoding='utf-8', index=False)

In [74]:
!ls

1_8호선_위치정보.csv  가중치_물류량2_clustering.csv
1_9호선_위치정보.csv  가중치_물류량.csv
9호선_위치정보.csv    서울교통공사_건물정보_20210228.csv
IC_정보.csv	      지하철역2.csv
IC정보.csv	      지하철역.csv
new_가중치1.csv       지하철역정보.csv
tmp.csv		      지하철정보.json
tmp_distance.csv      차량기지_위치데이터.csv
weight_logistic.csv   택배도착량.csv
가중치_물류량1.csv    택배발생량.csv


In [84]:
station = pd.read_csv('차량기지_위치데이터.csv')
station = station[['명칭', '위도', '경도']]
station

,명칭,위도,경도
0,군자차량사업소,37.55380,127.05452
1,신정차량사업소,37.51167,126.86293
2,지축차량사업소,37.65188,126.88697
3,수서차량사업소,37.48646,127.10377
4,창동차량사업소,37.63669,127.08010
5,고덕차량사업소,37.56350,127.17221
6,방화차량사업소,37.58579,126.81504
7,신내차량사업소,37.61175,127.10508
8,도봉차량사업소,37.70105,127.04814
9,천왕차량사업소,37.48120,126.84578


In [93]:
def cal_distance(df1: pd.DataFrame, df2: pd.DataFrame) -> pd.DataFrame:
  '''
  Input:
  df1: 차량 사업소(차량기지) -> 명칭, 위도, 경도
  df2: IC(inter connect) -> IC명, 위도, 경도

  Output:
  df: 차량 사업소-IC -> 차량 사업소 명칭, IC 명칭, 위도-경도에 대한 거리
  '''
  res = []
  for _, row in df1.iterrows():
    new_row = []
    new_row.append(row[0])
    for _, row2 in df2.iterrows():
      distance = haversine(row[1:3], row2[1:3], unit='km')
      res.append(new_row + [row2[0], distance])
  return pd.DataFrame(res, columns=['차량기지명', 'IC명', '거리(km)'])

In [96]:
highway_distance =\
  cal_distance(station, ic).sort_values('거리(km)').reset_index(drop=True)
highway_distance

,차량기지명,IC명,거리(km)
0,고덕차량사업소,강일IC,1.168301
1,고덕차량사업소,강일IC,1.259153
2,고덕차량사업소,상일IC,1.634431
3,고덕차량사업소,선동IC,1.672040
4,고덕차량사업소,상일IC,1.748786
...,...,...,...
935,천왕차량사업소,음성IC,87.728248
936,신정차량사업소,음성IC,88.717202
937,도봉차량사업소,음성IC,93.944064
938,방화차량사업소,음성IC,97.353536


In [104]:
highway_distance.to_csv('차량사업소_IC거리.csv', index=False)

In [100]:
highway_distance.groupby(['차량기지명', 'IC명']).mean()

거리(km)
차량기지명   IC명              
고덕차량사업소 가좌IC    44.675320
        강일IC     1.213727
        강촌IC    44.120965
        계양IC    37.229070
        광명역IC   31.160152
...                   ...
천왕차량사업소 판교IC    24.443571
        평촌IC    15.237157
        평택고덕IC  53.231217
        향남IC    41.463510
        화도IC    44.473608

[790 rows x 1 columns]

In [103]:
highway_distance.groupby(['차량기지명', 'IC명']).mean().sort_values(['차량기지명', '거리(km)'])

거리(km)
차량기지명   IC명             
고덕차량사업소 강일IC    1.213727
        선동IC    1.672040
        상일IC    1.691608
        토평IC    2.602116
        미사IC    2.658844
...                  ...
천왕차량사업소 서여주IC  68.089068
        남여주IC  70.388982
        강촌IC   74.153461
        여주IC   75.851662
        음성IC   87.728248

[790 rows x 1 columns]